## Homework 2

### 1. Generate a tree graph that represents flipping a coin 4 times,  let A be the event “the first outcome is tails” , B the event “ the second outcome is head” and C the event “the third outcome is tails” calculate 

p(AUBUC) = P (A U B U C) = P(A) + P(B) + P(C) - P(A ∩ B) - P(A ∩ C) - P(B ∩ C) + P(A ∩ B ∩ C)

### 2. From the Dataset Diabetes, construct contingency tables for the following variable combinations:

A: location Vs gender
B: Gender Vs frame
C: Gender Vs Age (Convert age to an discrete ordinal variable with three categories)
D: Cholesterol Vs Age (Convert age and cholesterol to an discrete ordinal variable with three categories)

calculate the joint and marginal probabilities, and from the above contingency tables choose 5 conditional probability examples with the probabilities calculations and one or two sentences explaining the results.

### 3. Baye’s Rule

Write a function in R that allows you to use the Baye’s rule for multiple events.

Use it to calculate the following problem:

Different isoforms of fundamental Hormones such as testosterone are relatively associated to behavioral differences in humans such as extreme aggression.  In a study where 75 % of the participants had the Isoform A, and 25 % had the isoform B. 54 people exhibited extreme aggression out 95  that had Isoform A, and 34 people with extreme aggression out of 90 that had the isoform B. 

Calculate the probability that someone with the isoform A exhibits extreme aggression. 
Calculate the probability that someone with the isoform B exhibits extreme aggression. 